In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import pandas as pd
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()
import tensorflow as tf
import tensorflow_hub as hub
from keras import backend as K
import keras.layers as layers

Using TensorFlow backend.


In [2]:

from keras.models import Model, load_model
import tensorflow.python.keras.engine

from keras.engine import Layer
import numpy as np

# Initialize session
sess = tf.Session()
K.set_session(sess)
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv('unprocessed_lyrics.csv')
df = df.groupby('Genre', group_keys=False).apply(lambda s: s.sample(5000, random_state=42)) #



In [3]:
possible_labels = df.Genre.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

df['label'] = df.Genre.replace(label_dict)

In [4]:
df

,SName,Lyric,Genre,Artist,lyric_length,label
2157,Family Affair,Refrain:. Let's get it crunkupon. We gon' have...,Hip Hop,Mary J. Blige,567,0
3187,Ransom (ft. Lil' Wayne),"Ransom,. . Yeah,. its Drizzy Baby. you already...",Hip Hop,Drake,967,0
9164,Close To Me,"T.O.S.. (50 Cent). Unstoppable, incredible, im...",Hip Hop,G-Unit,557,0
2145,Zone,"uhh, yea. uh uh uh. alright, well alright. . i...",Hip Hop,Drake,400,0
6442,"Why You Up In Here (feat. Ludacris, Git Fresh ...",Flo-Rida. Gucci!. Bird!. I done bought all thi...,Hip Hop,Flo Rida,524,0
...,...,...,...,...,...,...
31738,What do You Need?,What do you need from me tonight?. I feel you ...,Rock,Goo Goo Dolls,197,2
31425,Rebel Heart,"(R. Stewart, J. Golub, C. Kentis, C. Rojas). I...",Rock,Rod Stewart,412,2
35407,Before The Dawn,Meet me after dark again and I'll hold you. I ...,Rock,Evanescence,114,2
35281,Spanish is the Loving Tongue,"Broke my heart, lost my soul. Adios,mi cora so...",Rock,Bob Dylan,126,2


In [5]:
# confirm TensorFlow sees the GPU
from tensorflow.python.client import device_lib
assert 'GPU' in str(device_lib.list_local_devices())

#tf.config.list_physical_devices('GPU') 


In [6]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()) # list of DeviceAttributes



[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5285020765833077895
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 14408400334928113412
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 3472884504128192756
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14949928141
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18268486277284971855
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:1e.0, compute capability: 7.5"
]


In [7]:

df["label"] = LabelEncoder().fit_transform(df["Genre"])


In [8]:
#!pip install tensorflow_hub
#import tensorflow_hub as hub
#tf.compat.v1.disable_eager_execution()


#


embed = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [9]:
#import tensorflow as tf
import pandas as pd
import keras

from sklearn import preprocessing
from keras.utils import np_utils
import numpy as np


y = list(df['label'])
x = list(df['Lyric'])

le = preprocessing.LabelEncoder()
le.fit(y)

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.np_utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)


x_enc = x
y_enc = encode(le, y)

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(np.asarray(x_enc), np.asarray(y_enc), test_size=0.2, random_state=42)

In [11]:
x_train.shape


(12000,)

In [12]:
tf.get_logger().setLevel('ERROR')

from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

def ELMoEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

input_text = Input(shape=(1,), dtype=tf.string)

embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text) # elmo vectors are of size 1024. Our first layer (input layer)
dense = Dense(1024, activation='relu')(embedding) # second layer, (dense layer) each node is connected to all nodes of the preceding layer
pred = Dense(3, activation='softmax')(dense) # output layer
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=2, batch_size=8)
    model.save_weights('./elmo-model.h5')
    predicts = model.predict(x_test, batch_size=16)


##with tf.Session() as session:
#    K.set_session(session)
#    session.run(tf.global_variables_initializer())
#    session.run(tf.tables_initializer())
#    model.load_weights('./elmo-model.h5')  
    
    
y_test = decode(le, y_test)
y_preds = decode(le, predicts)


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:1e.0, compute capability: 7.5

Epoch 1/2
12000/12000 [==============================] - 2650s 221ms/step - loss: 0.8151 - acc: 0.6219
Epoch 2/2
12000/12000 [==============================] - 2425s 202ms/step - loss: 0.7579 - acc: 0.6562


In [13]:
from sklearn import metrics

print(metrics.confusion_matrix(y_test, y_preds))

print(metrics.classification_report(y_test, y_preds, target_names=label_dict))

from sklearn.metrics import accuracy_score

print("Accuracy of ELMO is:",accuracy_score(y_test,y_preds))

[[691 231 115]
 [ 75 549 346]
 [ 17 293 683]]
              precision    recall  f1-score   support

     Hip Hop       0.88      0.67      0.76      1037
         Pop       0.51      0.57      0.54       970
        Rock       0.60      0.69      0.64       993

    accuracy                           0.64      3000
   macro avg       0.66      0.64      0.65      3000
weighted avg       0.67      0.64      0.65      3000

Accuracy of ELMO is: 0.641
